In [4]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ipywidgets import interact, FloatSlider

# === Static surface parameters from MarketVisionStudio.py ===
strikes = np.array([600, 610, 620, 630, 640])     # Strike Prices
maturities = np.array([1/12, 3/12, 6/12, 1, 2])   # Time to Maturity in Years
static_vols = np.array([
    [28.0, 24.5, 22.0, 20.5, 19.5],
    [27.5, 24.0, 21.8, 20.3, 19.3],
    [27.0, 23.5, 21.5, 20.0, 19.0],
    [26.5, 23.0, 21.2, 19.8, 18.8],
    [26.0, 22.5, 21.0, 19.5, 18.5]
])  # Implied vols in %-points

# === Decomposition helper functions (all in %-points) ===
def apply_sticky_strike(surf, pct):
    return surf + pct

def apply_parallel_shift(surf, pts):
    return surf + pts

def apply_skew_gradient(surf, strikes, put_slope, call_slope):
    out = surf.copy()
    center = np.mean(strikes)
    for i, K in enumerate(strikes):
        weight = abs(K - center) / center
        if K < center:
            out[:, i] += put_slope * weight
        else:
            out[:, i] += call_slope * weight
    return out

def apply_convexity(surf, strikes, put_conv, call_conv):
    out = surf.copy()
    center = np.mean(strikes)
    half_width = (strikes[-1] - strikes[0]) / 2
    for i, K in enumerate(strikes):
        rel = abs(K - center) / half_width
        if K < center - half_width/2:    # deep put wing
            out[:, i] += put_conv * rel**2
        elif K > center + half_width/2:  # deep call wing
            out[:, i] += call_conv * rel**2
    return out

# === Plot function: side-by-side static vs decomposed ===
def plot_surface_comparison(
    sticky_strike_shift: float,
    parallel_shift:      float,
    put_skew:            float,
    call_skew:           float,
    put_convexity:       float,
    call_convexity:      float
):
    # Left: original static surface
    surf_static = static_vols

    # Right: apply decomposition to static_vols
    surf = static_vols.copy()
    surf = apply_sticky_strike(surf, sticky_strike_shift)
    surf = apply_parallel_shift(surf, parallel_shift)
    surf = apply_skew_gradient(surf, strikes, put_skew, call_skew)
    surf = apply_convexity(surf, strikes, put_convexity, call_convexity)

    # Build a 1x2 subplot of 3D surfaces
    fig = make_subplots(
        rows=1, cols=2,
        specs=[[{'type':'surface'}, {'type':'surface'}]],
        subplot_titles=('Original Surface', 'Decomposed Surface')
    )

    # Add static surface (no colorbar)
    fig.add_trace(
        go.Surface(
            x=strikes, y=maturities, z=surf_static,
            colorscale='Viridis', showscale=False
        ),
        row=1, col=1
    )

    # Add decomposed surface (with colorbar)
    fig.add_trace(
        go.Surface(
            x=strikes, y=maturities, z=surf,
            colorscale='Viridis'
        ),
        row=1, col=2
    )

    # Layout adjustments
    fig.update_layout(
        title_text='Volatility Surface: Static vs Decomposed',
        width=1200, height=600,
        margin=dict(l=50, r=50, t=80, b=50),
        scene=dict(
            xaxis_title='Strike Price',
            yaxis_title='Time to Maturity (yrs)',
            zaxis_title='Implied Vol (%)'
        ),
        scene2=dict(
            xaxis_title='Strike Price',
            yaxis_title='Time to Maturity (yrs)',
            zaxis_title='Implied Vol (%)'
        )
    )

    return fig

# === Interactive sliders ===
interact(
    plot_surface_comparison,
    sticky_strike_shift=FloatSlider(
        value=0.0, min=-5.0, max=5.0, step=0.1,
        description='Sticky Shift (%)'
    ),
    parallel_shift=FloatSlider(
        value=0.0, min=-5.0, max=5.0, step=0.1,
        description='Parallel Shift (%)'
    ),
    put_skew=FloatSlider(
        value=0.0, min=-10.0, max=10.0, step=0.1,
        description='Put Skew (%)'
    ),
    call_skew=FloatSlider(
        value=0.0, min=-10.0, max=10.0, step=0.1,
        description='Call Skew (%)'
    ),
    put_convexity=FloatSlider(
        value=0.0, min=-50.0, max=50.0, step=0.5,
        description='Put Convexity (%)'
    ),
    call_convexity=FloatSlider(
        value=0.0, min=-50.0, max=50.0, step=0.5,
        description='Call Convexity (%)'
    ),
)


interactive(children=(FloatSlider(value=0.0, description='Sticky Shift (%)', max=5.0, min=-5.0), FloatSlider(v…

<function __main__.plot_surface_comparison(sticky_strike_shift: float, parallel_shift: float, put_skew: float, call_skew: float, put_convexity: float, call_convexity: float)>